In [5]:
import torch
from db_helper_functions import get_stock_news_from_db
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)

c:\Users\vince\MADS\Milestone 2\w24-milestone2-team18-leevw-steveso-raulmart\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df = get_stock_news_from_db("AAPL")
df = df[~df.article.isnull()]
t_df = df.iloc[:100]
t_df

,id,ticker,title,url,article,date
0,4235,AAPL,"Microsoft, Alphabet, Meta, Apple, And Amazon L...",https://www.benzinga.com/news/earnings/22/07/2...,(Monday Market Open) Investors appear optimist...,2022-07-25
1,4376,AAPL,Benzinga Before The Bell: More Amazon Workers ...,https://www.benzinga.com/news/22/08/28532545/b...,CNBCBlackRock To Pledge A$1B In Australian Bat...,2022-08-17
2,3,AAPL,"Market Rebounds On Trade Optimism, Tech Bounce...",https://www.benzinga.com/node/12934767,A stronger-than-expected government report on ...,2019-01-04
3,4,AAPL,American Shippers And Carriers React To Signs ...,https://www.benzinga.com/node/12938837,The first week of 2019 saw three of America's ...,2019-01-07
4,5,AAPL,TD Ameritrade IMX Continued Its Dip In Decembe...,https://www.benzinga.com/node/12953011,"Declining for the third month in a row, TD Ame...",2019-01-08
...,...,...,...,...,...,...
95,132,AAPL,46 Stocks Moving In Wednesday's Mid-Day Session,https://www.benzinga.com/node/13642737,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-01
96,133,AAPL,Mid-Afternoon Market Update: Nu Skin Enterpris...,https://www.benzinga.com/node/13643990,"Toward the end of trading Wednesday, the Dow t...",2019-05-01
97,135,AAPL,60 Biggest Movers From Yesterday,https://www.benzinga.com/node/13649199,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-02
98,137,AAPL,"After Sliding On Powell's Words, Stocks Look F...",https://www.benzinga.com/node/13652243,The Fed threw one right down the middle late W...,2019-05-02


In [9]:
finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [11]:
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [12]:
start_pos = 512
end_pos = 512 * 2
raw_list_data_for_finbert = [x[start_pos:end_pos] for x in t_df.article]

In [13]:
finbert_mat = finbert_tokenizer(
    raw_list_data_for_finbert, padding=True, return_tensors="pt"
)

In [14]:
outputs = finbert_model(**finbert_mat)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5397,  2.6536, -0.6232],
        [ 0.1548,  1.4477, -2.0372],
        [ 2.0611, -1.5176, -1.7251],
        [-1.8773,  3.0955, -0.8240],
        [-1.8005,  2.9926, -0.8952],
        [ 1.8799, -1.6165, -1.3588],
        [ 1.6774, -1.8672, -0.6992],
        [-1.0231,  1.6744, -0.4548],
        [-1.2595, -0.3496,  2.2209],
        [-1.1794,  2.6108, -1.2826],
        [-1.3115,  2.6786, -1.2263],
        [-1.5596,  2.7557, -0.8460],
        [-1.2293,  2.5503, -1.0207],
        [-1.5249,  2.3742, -0.4214],
        [ 1.8723, -0.9890, -1.9238],
        [ 1.4085, -0.9537, -1.2859],
        [ 1.1091, -0.1662, -1.7661],
        [ 1.7246, -1.4443, -1.1983],
        [-0.5723,  1.9858, -1.4996],
        [-0.0505, -1.4916,  1.7669],
        [-1.0068, -0.5763,  2.1593],
        [-1.1289,  1.2519,  0.4317],
        [-1.5063,  2.6327, -0.8569],
        [ 0.3653, -2.0091,  1.5959],
        [-0.8133,  1.4951, -0.3326],
        [-1.3708,  2.5284, -0.84

In [15]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [16]:
t_df[["positive", "negative", "neutral"]] = predictions.tolist()
t_df

C:\Users\vince\AppData\Local\Temp\ipykernel_17004\1623899634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df[["positive", "negative", "neutral"]] = predictions.tolist()
C:\Users\vince\AppData\Local\Temp\ipykernel_17004\1623899634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_df[["positive", "negative", "neutral"]] = predictions.tolist()
C:\Users\vince\AppData\Local\Temp\ipykernel_17004\1623899634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,id,ticker,title,url,article,date,positive,negative,neutral
0,4235,AAPL,"Microsoft, Alphabet, Meta, Apple, And Amazon L...",https://www.benzinga.com/news/earnings/22/07/2...,(Monday Market Open) Investors appear optimist...,2022-07-25,0.014338,0.949808,0.035853
1,4376,AAPL,Benzinga Before The Bell: More Amazon Workers ...,https://www.benzinga.com/news/22/08/28532545/b...,CNBCBlackRock To Pledge A$1B In Australian Bat...,2022-08-17,0.210314,0.766196,0.023490
2,3,AAPL,"Market Rebounds On Trade Optimism, Tech Bounce...",https://www.benzinga.com/node/12934767,A stronger-than-expected government report on ...,2019-01-04,0.951845,0.026568,0.021587
3,4,AAPL,American Shippers And Carriers React To Signs ...,https://www.benzinga.com/node/12938837,The first week of 2019 saw three of America's ...,2019-01-07,0.006743,0.973923,0.019334
4,5,AAPL,TD Ameritrade IMX Continued Its Dip In Decembe...,https://www.benzinga.com/node/12953011,"Declining for the third month in a row, TD Ame...",2019-01-08,0.008055,0.972029,0.019917
...,...,...,...,...,...,...,...,...,...
95,132,AAPL,46 Stocks Moving In Wednesday's Mid-Day Session,https://www.benzinga.com/node/13642737,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-01,0.946302,0.018754,0.034944
96,133,AAPL,Mid-Afternoon Market Update: Nu Skin Enterpris...,https://www.benzinga.com/node/13643990,"Toward the end of trading Wednesday, the Dow t...",2019-05-01,0.954219,0.021828,0.023953
97,135,AAPL,60 Biggest Movers From Yesterday,https://www.benzinga.com/node/13649199,"Loading...Loading...Gainers\nARCA biopharma, I...",2019-05-02,0.944363,0.018706,0.036930
98,137,AAPL,"After Sliding On Powell's Words, Stocks Look F...",https://www.benzinga.com/node/13652243,The Fed threw one right down the middle late W...,2019-05-02,0.034162,0.932216,0.033622


In [26]:
idx_positive = t_df["positive"].idxmax()
idx_negative = t_df["negative"].idxmax()
idx_neutral = t_df["neutral"].idxmax()

print(f"Most positive: (positive: {t_df.loc[idx_positive, 'positive']}, negative: {t_df.loc[idx_positive, 'negative']}, neutral: {t_df.loc[idx_positive, 'neutral']}): {t_df.loc[idx_positive, 'article']}")
print(f"Most negative: (positive: {t_df.loc[idx_negative, 'positive']}, negative: {t_df.loc[idx_negative, 'negative']}, neutral: {t_df.loc[idx_negative, 'neutral']}): {t_df.loc[idx_negative, 'article']}")
print(f"Most neutral: (positive: {t_df.loc[idx_neutral, 'positive']}, negative: {t_df.loc[idx_neutral, 'negative']}, neutral: {t_df.loc[idx_neutral, 'neutral']}): {t_df.loc[idx_neutral, 'article']}")


Most positive: (positive: 0.9542192220687866, negative: 0.021827759221196175, neutral: 0.023953113704919815): Toward the end of trading Wednesday, the Dow traded down 0.1 percent to 26565.75 while the NASDAQ climbed 0.15 percent to 8107.29. The S&P also fell, dropping 0.18 percent to 2940.45.
Leading and Lagging SectorsWednesday afternoon, the information technology shares rose 0.6 percent. Meanwhile, top gainers in the sector included  Nanometrics Incorporated  NANO up 24 percent, and  Rogers Corporation  ROG up 16 percent.In trading on Wednesday, energy shares fell 1.2 percent.
Top HeadlineApple Inc. AAPL reported better-than-expected earnings for its second quarter.Earnings came in at $2.46 per share, beating estimates by 10 cents. Sales came in at $58 billion, beating estimates by $560 million.The company issued strong third-quarter sales guidance of $52.5 billion-$54.5 billion; Estimates called for $51.9 billion. Equities Trading UPEnphase Energy, Inc.  ENPH shares got a boost, sh

In [34]:
from db_helper_functions import get_stock_news_with_bertopic_sentiment_scores_from_db
import numpy as np
from constants import SHARED_RANDOM_STATE
from db_helper_functions import get_stock_news_from_db
from text_cleaning_functions import clean_text
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from finbert_embedding.embedding import FinbertEmbedding

bertopic_df = get_stock_news_with_bertopic_sentiment_scores_from_db("AAPL") 
# tf_vectorizer = CountVectorizer(stop_words="english", ngram_range=(1, 3))
# finbert_embedding = FinbertEmbedding()
# bert_topic_model = BERTopic(
#     embedding_model=finbert_embedding,
#     language="english",
#     calculate_probabilities=True,
#     nr_topics=10,
# )
# topics, probs = bert_topic_model.fit_transform(list(bertopic_df["article"]))
# freq = bert_topic_model.get_topic_info()
# freq.head(10)
bertopic_df

,id,ticker,title,url,article,date,positive,negative,neutral
0,1,AAPL,58 Biggest Movers From Yesterday,https://www.benzinga.com/node/12932897,Loading...Loading...Gainers\nOHR Pharmaceutica...,2019-01-04,0.464299,0.192475,0.343222
1,2,AAPL,Apple Lowers Revenue Expectations As China's M...,https://www.benzinga.com/node/12933509,Among news of poor performance in the Chinese ...,2019-01-04,0.543883,0.162925,0.293192
2,3,AAPL,"Market Rebounds On Trade Optimism, Tech Bounce...",https://www.benzinga.com/node/12934767,A stronger-than-expected government report on ...,2019-01-04,0.379343,0.235905,0.384743
3,4,AAPL,American Shippers And Carriers React To Signs ...,https://www.benzinga.com/node/12938837,The first week of 2019 saw three of America's ...,2019-01-07,0.374402,0.243911,0.381680
4,5,AAPL,TD Ameritrade IMX Continued Its Dip In Decembe...,https://www.benzinga.com/node/12953011,"Declining for the third month in a row, TD Ame...",2019-01-08,0.256786,0.369308,0.373888
...,...,...,...,...,...,...,...,...,...
5370,5373,AAPL,"Tesla, Amazon, Apple, Alibaba, Microsoft: Why ...",https://www.benzinga.com/news/23/01/30287594/t...,Major Wall Street indices closed in the green ...,2023-01-04,0.450044,0.187260,0.362690
5371,5374,AAPL,Microsoft Finds Apple Gatekeeper's 'Achilles' ...,https://www.benzinga.com/news/22/12/30141117/m...,Microsoft Corporation MSFT shared a macOS secu...,2022-01-09,0.424009,0.201395,0.374600
5372,5375,AAPL,"Facebook, Amazon, Apple, Google Stamping Out R...",https://www.benzinga.com/government/20/10/1781...,The dominance of large technology companies li...,2020-10-07,0.318888,0.303523,0.377590
5373,5376,AAPL,Apple's New Patents Indicate Macs Will Be Able...,https://www.benzinga.com/news/21/01/19032246/a...,Apple Inc AAPL has secured patents related to ...,2021-01-06,0.379527,0.216145,0.404337
